## 4.2 Das Gram-Schmidt-Verfahren

In [ ]:
import numpy as np
from scripts.LR_Zerlegung import rueckwaerts_einsetzen

Wir implementieren das klassische Gram-Schmidt-Verfahren, welches in Satz 4.6 beschrieben ist.

In [ ]:
def gram_schmidt(A):
    n = A.shape[0]
    Q = np.zeros_like(A)
    
    for i in range(n):
        Q[:, i] = A[:, i]
        for j in range(i):
            Q[:, i] -= np.inner(A[:,i], Q[:, j]) * Q[:, j]
        Q[:, i] /= np.linalg.norm(Q[:, i])
     
    return Q

*Ergänzende  Einzelheiten zum Code*
- Anstelle die Vektoren $\tilde{q}_i$ einzeln zu speichern, speichern $\tilde{q}_i$ in der i-ten Spalte von der Matrix `Q`. 
- Die `numpy` Funktion `inner` gibt uns eine effiziente Implementierung des Skalarproduktes für zwei arrays.
- Die `numpy` Funktion `linalg.norm` gibt uns eine effiziente Implementierung der 2-Norm. Über das Argument `ord` lassen sich damit auch andere Normen von Vektoren und Matrizen berechnen.

#### Beispiel 4.7 (Gram-Schmidt)
Wir wenden unsere Implementierung nun auf die 3x3 Hilbert-Matrix
$$
A = 
\begin{pmatrix} 1 & 1/2 & 1/3\\ 1/2 & 1/3 & 1/4\\1/3 & 1/4 & 1/5\end{pmatrix}
$$
an. Um die Effekte der endlichen Arithmetik zu verdeutlichen, nehmen wir dabei `half` Gleitkommazahlen.

In [ ]:
A = np.array([[1,     1 / 2, 1 / 3],
              [1 / 2, 1 / 3, 1 / 4],
              [1 / 3, 1 / 4, 1 / 5]], dtype=np.half)
Q = gram_schmidt(A)
print(Q)

Nun überprüfen wir inwiefern das Ergebnis wirklich eine orthogonale Matrix ist. Um zwei zweidimensionale `numpy` wie Matrizen zu multiplizieren, müssen wir den `@` Operator verwenden. Der `*` Operator erzeugt das komponentenweise Produkt von zwei arrays.

In [ ]:
print(Q @ Q.transpose())
err_ort = np.linalg.norm(Q @ Q.transpose() - np.identity(Q.shape[0]), ord=2)
print(f'||Q * Q^T - I||_2 = {err_ort}')

#### Beispiel 4.8 (QR-Zerlegung mit Gram-Schmidt)

Jetzt können wir das obige Gram-Schmidt-Verfahren anpassen um eine QR-Zerlegung der Matrix zu berechnen.

In [ ]:
def qr_gram_schmidt(A):
    n = A.shape[0]
    Q, R = np.zeros_like(A), np.zeros_like(A)
    
    for i in range(n):
        Q[:, i] = A[:, i]
        for j in range(i):
            Q[:, i] -= np.inner(A[:,i], Q[:, j]) * Q[:, j]
        Q[:, i] /= np.linalg.norm(Q[:, i])
        for j in range(i, n):
            R[i, j] = np.inner(Q[:, i], A[:, j])
    return Q, R

Wir testen dies nun an dem linearen Gleichungssystem $Ax=v$ mit
$$
A = 
\begin{pmatrix} 1 & 1 & 1\\ 0.01 & 0 & 0.01\\ 0 & 0.01 & 0.01\end{pmatrix}
\qquad
b =
\begin{pmatrix} 1\\ 0\\ 0.02\end{pmatrix}
$$
welches die exakte Lösung
$$
x =
\begin{pmatrix} -1\\ 1\\ 1\end{pmatrix}
$$
hat. Um wieder die Effekte der endlichen Arithmetik zu verdeutlichen, nehmen wir dabei auch `half` Gleitkommazahlen.

In [ ]:
A = np.array([[1,    1,    1   ],
              [0.01, 0,    0.01],
              [0,    0.01, 0.01]], dtype=np.half)
b = np.array([1, 0, 0.02], dtype=np.half)
x_ex = np.array([-1, 1, 1])

Q, R = qr_gram_schmidt(A)

b2 = np.dot(Q.transpose(), b)
x = rueckwaerts_einsetzen(R, b2)

print('x =', x)
print('x_ex = ', x_ex)

Das ergibt den relativen Fehler

In [ ]:
rel_err = np.linalg.norm(x - x_ex) / np.linalg.norm(x_ex)
print(f'||x - x_ex|| / ||x_ex|| = {rel_err}')

Die Abweichung der Orthogonalität in der Frobenius-Norm ist dabei

In [ ]:
err_ort = np.linalg.norm(Q @ Q.transpose() - np.identity(Q.shape[0]), ord=2)
print(f'||Q * Q^T - I||_2 = {err_ort}')

#### Beispiel 4.10 (Modifiziertes Gram-Schmidt-Verfahren)

Da das Verfahren in dieser Form nicht besonders stabil ist, implementieren wir nun das modifizierte Gram-Schmidt-Verfahren.

In [ ]:
def mod_gram_schmidt(A):
    n = A.shape[0]
    Q = np.zeros_like(A)
    
    for i in range(n):
        Q[:, i] = A[:, i]
        for j in range(i):
            Q[:, i] -= np.inner(Q[:, i], Q[:, j]) * Q[:, j]
        Q[:, i] /= np.linalg.norm(Q[:, i])
    return Q 

Angewandt auf die 3x3 Hilbert-Matrix ergibt bei `half` Gleitkommadarstellung

In [ ]:
A = np.array([[1,     1 / 2, 1 / 3],
              [1 / 2, 1 / 3, 1 / 4],
              [1 / 3, 1 / 4, 1 / 5]], dtype=np.half)

Q = mod_gram_schmidt(A)
err_ort = np.linalg.norm(Q @ Q.transpose() - np.identity(Q.shape[0]), ord=2)
print(f'Q =\n{Q}')
print(f'||Q * Q^T - I||_2 = {err_ort}')

Die Orthogonalität hat sich somit um etwa den Faktor 3.7 verbessert. Wir können diese Verfahren nun also auch dazu verwenden um eine QR-Zerlegung zu berechnen.

In [ ]:
def qr_mod_gram_schmidt(A):
    n = A.shape[0]
    Q, R = np.zeros_like(A), np.zeros_like(A)
    
    for i in range(n):
        Q[:, i] = A[:, i]
        for j in range(i):
            Q[:, i] -= np.inner(Q[:, i], Q[:, j]) * Q[:, j]
        Q[:, i] /= np.linalg.norm(Q[:, i])
        for j in range(i, n):
            R[i, j] = np.inner(Q[:, i], A[:, j])
    return Q, R

Wenn wir dies nun anwenden um das obige lineare Gleichungssystem zu lösen, sehen wir

In [ ]:
A = np.array([[1,    1,    1   ],
              [0.01, 0,    0.01],
              [0,    0.01, 0.01]], dtype=np.half)
b = np.array([1, 0, 0.02], dtype=np.half)
x_ex = np.array([-1, 1, 1])

Q, R = qr_mod_gram_schmidt(A)
b3 = np.dot(Q.transpose(), b)
x2 = rueckwaerts_einsetzen(R, b3)
print(f'x = {x2}')

Diese Lösung ist weiterhin nicht sehr genau. Wenn wir dies quantifizieren, sehen wir

In [ ]:
rel_err = np.linalg.norm(x2 - x_ex) / np.linalg.norm(x_ex)
print(f'||x - x_ex|| / ||x_ex|| = {rel_err}')

err_ort = np.linalg.norm(Q @ Q.transpose() - np.identity(Q.shape[0]), ord=2)
print(f'||Q * Q^T - I||_2 = {err_ort}')

Obwohl wir die Orthogonalität der Matrix $Q$ um den Faktor 70 verbessert haben, ist der relative Fehler der Lösung nicht einmal halbiert worden.